# Finite Difference Method for the Heat Equation on the Unit Interval

Here we solve the wave equation $\frac{\partial \, u}{\partial t} = c^2 \frac{\partial^2 \, u}{\partial x^2}$ on the spatial domain $x \in \Omega=(0,1)$ and time domain $t \in [0,T]$ supplied with homogeneous Dirichlet boundary conditions and initial conditions for displacement and velocity. For the spatial discretization, the finite difference method on an uniform mesh is employed. For the time discretization, a fixed time step is employed.  

## Import Packages

In [2]:
using LinearAlgebra
using DifferentialEquations
using SparseArrays
using Plots
using BenchmarkTools 

Documentation of DifferentialEquations.jl  
<ul>
<li> <a href="https://diffeq.sciml.ai/dev/tutorials/ode_example/"> Examples </a>   
<li> <a href="https://diffeq.sciml.ai/stable/"> Documentation of DifferentialEquations.jl on Github </a>  
<li> <a href="https://diffeq.sciml.ai/dev/features/performance_overloads/#Function-Type-Definitions"> Specifying Jacobian types using DifferentialEquations.jl </a>   
</ul>   

## References